# Python Solutions to Tutorial 07


In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stat
import matplotlib.pyplot as plt

#url = 'https://raw.githubusercontent.com/APS1040/Tutorials/main/Tutorial07_data.csv'
#df = pd.read_csv(url)

df = pd.read_csv('Tutorial07_data.csv')
df.head()

,Sample,Vol
0,1,6
1,1,9
2,1,10
3,1,15
4,2,10


## 1.	A high-level voltage power supply should have a nominal output voltage of 350 V. A sample of four units is selected each day and tested for process control purposes. The data shown in the dataset give the difference between the observed reading on each unit and the nominal voltage times ten; that is, Vol_i= (observed voltage on unit i - 350)*10. Set up X-bar and R charts. Is the process in statistical control?

In [2]:
m = 20 
R_accum = 0 
Rs = [] 

for n in range (1,m+1):        
    max = df.loc[df['Sample']==n,'Vol'].max()
    min = df.loc[df['Sample']==n,'Vol'].min()
    R = max-min
    R_accum = R_accum + R
    Rs.append(R)
CL_R = R_accum/m
print(CL_R)
UCL_R = 2.282 * CL_R 
LCL_R = 0
print(LCL_R, UCL_R)

6.25
0 14.2625


In [3]:
for n in range(1,m+1):
    if Rs[n-1] >= UCL_R: 
         print(n, 'is out of bounds')

In [4]:
m = 20
x_bar_accum = 0
x_bars = []

for n in range (1,m+1):        
    x_bar = df.loc[df['Sample']==n,'Vol'].mean()
    x_bars.append(x_bar)
    x_bar_accum = x_bar_accum+x_bar

CL_x_bar = x_bar_accum/m
print(CL_x_bar)

UCL_x_bar = CL_x_bar + 0.729 * CL_R # A2
LCL_x_bar = CL_x_bar - 0.729 * CL_R # A2

print(UCL_x_bar, LCL_x_bar)

10.325
14.881249999999998 5.76875


In [5]:
for n in range(1,m+1):
    if (x_bars[n-1] > UCL_x_bar or x_bars[n-1] < LCL_x_bar ): 
         print(n, 'is out of bounds')

It is in control

## 2.	What is the estimate of the process mean?

In [6]:
print(CL_x_bar)

10.325


## 3.	What is the estimate of the process standard deviation?

In [7]:
sigma = CL_R/2.059 #R_bar/d2
print(sigma)

3.035454103933948


## 4.	What are the Natural Tolerance Limits for this process?

In [8]:
ul = CL_x_bar+3*sigma
ll = CL_x_bar-3*sigma
print(ll,ul)

1.2186376881981538 19.431362311801845


## 5.	If the specifications are USL = 352V, LSL = 348V, what is the value of Cp? 

In [9]:
cp = (20-(-20))/(6*sigma)
print(cp)

2.196266666666667


## 6.	Is the basic capability requirement met (i.e., is the process capable)? Explain.

Yes it is met because cp > 1

## 7.	What is the process fallout? 

In [10]:
# in ppm
(stat.norm.cdf(-20, CL_x_bar,sigma)+1-stat.norm.cdf(20, CL_x_bar,sigma))*(10**6)

717.9592415318403

## 8.	What is the value of CPK? 

In [11]:
cpu = (20-CL_x_bar)/(3*sigma)
cpl = (CL_x_bar-(-20))/(3*sigma)
cpk = min(cpu,cpl)
print(cpk)

1.0624440000000002


## 9.	What is the best approximation of the process fallout if you calculate using CPK? 

In [12]:
# approximate using c_pk
(1-stat.norm.cdf(3*cpk, 0,1))*(10**6)

717.9592415318403

## 10.	Is the process centred? Explain.

No, a process is centered if cpl=cpu

In [13]:
print(cpl, cpu)
print(cpl == cpu)

3.3300893333333335 1.0624440000000002
False


## 11.	What is the value of CPM? 

In [14]:
t = (20+(-20))/2
cpm = cp/(np.sqrt(1+( (CL_x_bar-t)/sigma )**2 ))
print(cpm)

0.6194663187960908
